In [1]:
import pandas as pd

### Read in data

In [2]:
books = pd.read_csv("Data/goodbooks-10k/books.csv", error_bad_lines = False)
books.head()

/Users/melanieackerman/opt/anaconda3/envs/metis/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [3]:
books.shape

(10000, 23)

In [4]:
ratings = pd.read_csv("Data/goodbooks-10k/ratings.csv")
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [5]:
ratings.shape

(5976479, 3)

In [6]:
tags = pd.read_csv("Data/goodbooks-10k/tags.csv")
tags.head(30)

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-
5,5,--166-
6,6,--17-
7,7,--19-
8,8,--2-
9,9,--258-


In [7]:
booktags = pd.read_csv("Data/goodbooks-10k/book_tags.csv")
booktags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


### Clean data: remove duplicates

In [8]:
ratings = ratings.sort_values('user_id')
ratings.head()
ratings.shape

(5976479, 3)

In [9]:
ratings.drop_duplicates(subset = ['user_id', 'book_id'], keep=False, inplace=True)
ratings.shape

(5976479, 3)

In [10]:
# ratings df is too big to convert to excel file. Splitting into 6 chunks
import numpy as np
ratings_split = np.array_split(ratings, 6)

In [11]:
print(books.shape)
books.drop_duplicates(subset = 'original_title', keep=False, inplace=True)
print(books.shape)

(10000, 23)
(9151, 23)


In [12]:
print(booktags.shape)
booktags.drop_duplicates(subset = ['tag_id', 'goodreads_book_id'], keep=False, inplace=True)
print(booktags.shape)

(999912, 3)
(999896, 3)


In [13]:
print(tags.shape)
tags.drop_duplicates(subset = 'tag_id', keep=False, inplace=True)
print(tags.shape)

(34252, 2)
(34252, 2)


### Clean Data: tag categories
Coding genres since tags are created by users. Credit for this approach to @philispp on kaggle via @niharika41298 on kaggle

In [14]:
joint_tags = pd.merge(booktags, tags, left_on='tag_id', right_on='tag_id', how='inner')

In [15]:
joint_tags.head()

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


In [16]:
tagnames = joint_tags.groupby('tag_name').count()
tagnames = tagnames.sort_values(by='count', ascending=False)
tagnames

,goodreads_book_id,tag_id,count
tag_name,,,
to-read,9983,9983,9983
favorites,9881,9881,9881
owned,9856,9856,9856
books-i-own,9799,9799,9799
currently-reading,9776,9776,9776
...,...,...,...
hs,1,1,1
hrabal,1,1,1
hq-manga,1,1,1


In [17]:
genres = ["Art", "Biography", "Business", "Chick Lit", "Children's", "Christian", "Classics", 
          "Comics", "Contemporary", "Cookbooks", "Crime", "Ebooks", "Fantasy", "Fiction", 
          "Gay and Lesbian", "Graphic Novels", "Historical Fiction", "History", "Horror", 
          "Humor and Comedy", "Manga", "Memoir", "Music", "Mystery", "Nonfiction", "Paranormal", 
          "Philosophy", "Poetry", "Psychology", "Religion", "Romance", "Science", "Science Fiction", 
          "Self Help", "Suspense", "Spirituality", "Sports", "Thriller", "Travel", "Young Adult"]

for i in range(len(genres)):
    genres[i] = genres[i].lower()

In [18]:
new_tags = tagnames[tagnames.index.isin(genres)]

In [19]:
# Obtain books with the tags from new_tags
newbooktags = joint_tags[joint_tags.tag_name.isin(genres)]
newbooktags.head()

,goodreads_book_id,tag_id,count,tag_name
9983,1,11305,37174,fantasy
9984,2,11305,3441,fantasy
9985,3,11305,47478,fantasy
9986,5,11305,39330,fantasy
9987,6,11305,38378,fantasy


In [20]:
newbooktags.tail()

,goodreads_book_id,tag_id,count,tag_name
761405,15811496,8150,50,cookbooks
761406,16182601,8150,67,cookbooks
761407,17333302,8150,7,cookbooks
761408,18167006,8150,9,cookbooks
761409,23398625,8150,13,cookbooks


### Save to excel files

In [34]:
with pd.ExcelWriter('Data/books_master.xlsx') as writer:
    books.to_excel(writer, sheet_name='books')
    tags.to_excel(writer, sheet_name='tags')
    booktags.to_excel(writer, sheet_name='booktags')
    joint_tags.to_excel(writer, sheet_name='joint_tags')
    new_tags.to_excel(writer, sheet_name='new_tags')
    ratings_split[0].to_excel(writer, sheet_name='ratings1')
    ratings_split[1].to_excel(writer, sheet_name='ratings2')
    ratings_split[2].to_excel(writer, sheet_name='ratings3')
    ratings_split[3].to_excel(writer, sheet_name='ratings4')
    ratings_split[4].to_excel(writer, sheet_name='ratings5')
    ratings_split[5].to_excel(writer, sheet_name='ratings6')
    

In [35]:
books.to_excel('Data/books_main.xlsx', sheet_name='books')

In [36]:
tags.to_excel('Data/tags_main.xlsx', sheet_name='tags')
booktags.to_excel('Data/booktags_main.xlsx', sheet_name='booktags')
joint_tags.to_excel('Data/joint_tags_main.xlsx', sheet_name='joint_tags')
new_tags.to_excel('Data/new_tags_main.xlsx', sheet_name='new_tags')

In [37]:
ratings_split[0].to_excel('Data/ratings1.xlsx', sheet_name='ratings1')
ratings_split[1].to_excel('Data/ratings2.xlsx', sheet_name='ratings2')
ratings_split[2].to_excel('Data/ratings3.xlsx', sheet_name='ratings3')
ratings_split[3].to_excel('Data/ratings4.xlsx', sheet_name='ratings4')
ratings_split[4].to_excel('Data/ratings5.xlsx', sheet_name='ratings5')
ratings_split[5].to_excel('Data/ratings6.xlsx', sheet_name='ratings6')

In [21]:
newbooktags.to_excel('Data/new_book_tags_main.xlsx', sheet_name='new_book_tags')

In [22]:
newbooktags.shape

(60573, 4)